# csapp学习

## 第二章: 信息的表示与处理

### d2h.pl

In [ ]:
!usr/local/bin/perl
# 十进制2十六进制

# for ($i = 0; $i < @ARGV; $i++) {
#     printf("%d\t= 0x%x\n", $ARGV[$i], $ARGV[$i]);
# }

# 十六进制2十进制

for ($i = 0; $i < @ARGV; $i++) {
    $val = hex($ARGV[$i]);
    printf("0x%x\t= %d\n", $val, $val);
}

### show_bytes.c

In [10]:
#include <stdio.h>

typedef unsigned char* byte_pointer;

void show_bytes(byte_pointer start, int len) {
  for (int i = 0; i < len; i++) {
    printf("%.2x", start[i]);
  }
  printf("\n");
}

void show_int(int x) { show_bytes((byte_pointer)&x, sizeof(int)); }

void show_float(float x) { show_bytes((byte_pointer)&x, sizeof(float)); }

void show_pointer(void* x) { show_bytes((byte_pointer)&x, sizeof(void*)); }

void test_show_bytes(int val) {
  int ival = val;
  float fval = (float)val;
  int* pval = &val;
  show_int(ival);
  show_float(fval);
  show_pointer(pval);
}

int main() {
  short sx = -12345;
  unsigned uy = sx;

  printf("sx = %d:\t", sx);
  show_bytes((byte_pointer)&sx, sizeof(short));

  printf("uy = %u:\t", uy);
  show_bytes((byte_pointer)&uy, sizeof(unsigned));
}


sx = -12345:	c7cf
uy = 4294954951:	c7cfffff


### bit.c

In [ ]:
#include <stdbool.h>  // 定义 bool, true, false
#include <stdio.h>

// // 将m为1的位设置到x 1
// int bis(int x, int m);
// // 将m为1的位设置到x 0
// int bic(int x, int m);

// int bool_xor(int x, int y) {
//   int res = bis(bic(x, y), bic(y, x));
//   return res;
// }

int get_low_eight(int x) {
  int mask = 0xFF;
  return x & mask;
}

/**
 * @brief 保留最低位,其余位取反
 * @note 值得学习的思路,任何^0都是本身
 */
int save_low_eight_not(int x) {
  int mask = ~0xFF;
  return x ^ mask;
}

int set_low_eight_one(int x) {
  int mask = 0xFF;
  return x | mask;
}

bool same(int x, int y) { return !(x ^ y); }

### expand_num.c

In [14]:
#include <limits.h>
#include <stdio.h>

/**
 * @note 提取低8位的值 0-255的整数
 */
int fun1(unsigned word) { return (int)((word << 24) >> 24); }

/**
 * @note 提取低8位的值且符号拓展 -128-127的整数
 */
int fun2(unsigned word) { return ((int)word << 24) >> 24; }

int main() {
  int x = -1;
  unsigned u = 2147483648;
  printf("x = %u,  = %d\n", x, x);
  printf("u = %u,  = %d\n", u, u);
}

x = 4294967295,  = -1
u = 2147483648,  = -2147483648


### reverse.c 用位运算技巧swap

In [ ]:
#include <stdio.h>

void inplace_swap(int* x, int* y) {
  *y = *x ^ *y;
  *x = *x ^ *y;
  *y = *x ^ *y;
}

void reverse_array(int a[], int cnt) {
  int last = cnt - 1;
  for (int i = 0; i <= last; i++, last--) {
    if (i == last) {
      break;
    }
    inplace_swap(&a[i], &a[last]);
  }
}

### 无符号的坑

In [5]:
#include <stdio.h>
#include <string.h>

float sum_elements(float a[], unsigned len){
  int i = 0;
  float result = 0;
  // 当len=0这里的len-1会变成U_MAX，使得条件一直成立，修复把函数声明的unsigned换int
  for(i = 0; i <= len -1 ; i++){
    result += a[i];
  }
  return result;
}

/**
 * @note 如果直接用减法,当s比t短的时候,unsigned int
 * 减出来的结果是个很大的unsigned数
 */
int strlonger(char* s, char* t) {
  // 现在这个是可靠的
  return strlen(s) > strlen(t);
}

int main() {
  float a[] = {0.1, 0.6, 1.5};
  unsigned len = 0;
  float ans = sum_elements(a, len);
  printf("%f", ans);
}

[C kernel] Executable exited with code -11

### 整数运算

In [11]:
#include <limits.h>
#include <stdio.h>

int uadd_ok(unsigned x, unsigned y) { return x <= UINT_MAX - y; }

int tadd_ok(int x, int y) {
  int sum = x + y;
  return !((x >= 0 && y >= 0 && sum < 0) || (x < 0 && y < 0 && sum >= 0));
}

int tsub_ok(int x, int y) {
  int diff = x - y;
  return !((x >= 0 && y < 0 && diff < 0) || (x < 0 && y >= 0 && diff > 0));
}

// /**
//  * @brief v1
//  */
// int tmult_ok(int x, int y) {
//   int p = x * y;
//   return !x || p / x == y;
// }

/**
 * @brief v2
 */
int tmult_ok(int x, int y) {
  long long mult = (long long)x * y;
  return mult == (int)mult;
}

int main() {
  unsigned x = 100, y = 1;
  int uans = uadd_ok(x, y);

  int a = INT_MIN, b = 1;
  int tans = tadd_ok(a, b);

  int c = 0, d = INT_MIN;
  int sans = tsub_ok(c, d);

  int e = 9594988, f = 525;
  int mans = tmult_ok(e, f);
  printf("uans = %d\n", uans);
  printf("tans = %d\n", tans);
  printf("sans = %d\n", sans);
  printf("mans = %d\n", mans);
}

uans = 1
tans = 1
sans = 0
mans = 0


### 补码的除法

In [26]:
#include <limits.h>
#include <stdio.h>

int div16(int x) {
  int bias = (x >> 31) & 0xF;
  printf("bias = %d\n", bias);
  return (x + bias) >> 4;
}

int main() {
  int ans = div16(-31);
  printf("ans = %d\n", ans);
}

bias = 15
ans = -1


### 表示无穷大

In [9]:
#include <stdio.h>

#define POS_INFINITY 1e400
#define NEG_INFINITY (-POS_INFINITY)
#define NEG_ZERO (-1.0/POS_INFINITY)

int main() {
  double ans = NEG_ZERO;
  printf("ans = %f\n", ans);
}

/tmp/tmp40f7mg00.c: In function ‘main’:
/tmp/tmp40f7mg00.c:8:3: warning: floating constant exceeds range of ‘double’ [-Woverflow]
    8 |   double ans = NEG_ZERO;
      |   ^~~~~~


ans = -0.000000
